In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
from astropy.table import Table
# from scipy.spatial import distance
# from scipy import spatial
# from scipy.spatial import KDTree
import time
import matplotlib.path as mpltPath
from pandas import DataFrame, read_csv
import pandas as pd 
from mpl_toolkits.mplot3d import Axes3D
import copy

In [5]:
file = r'Gaia_plx7_dr2.csv'
df = pd.read_csv(file)
df2 = df.columns.get_values()
df2.tolist()
#print('Succesfully Read in the Data')

/astromake/opt/python/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (96,122,130) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['Unnamed: 0',
 'solution_id',
 'designation',
 'source_id',
 'random_index',
 'ref_epoch',
 'ra',
 'ra_error',
 'dec',
 'dec_error',
 'parallax',
 'parallax_error',
 'parallax_over_error',
 'pmra',
 'pmra_error',
 'pmdec',
 'pmdec_error',
 'ra_dec_corr',
 'ra_parallax_corr',
 'ra_pmra_corr',
 'ra_pmdec_corr',
 'dec_parallax_corr',
 'dec_pmra_corr',
 'dec_pmdec_corr',
 'parallax_pmra_corr',
 'parallax_pmdec_corr',
 'pmra_pmdec_corr',
 'astrometric_n_obs_al',
 'astrometric_n_obs_ac',
 'astrometric_n_good_obs_al',
 'astrometric_n_bad_obs_al',
 'astrometric_gof_al',
 'astrometric_chi2_al',
 'astrometric_excess_noise',
 'astrometric_excess_noise_sig',
 'astrometric_params_solved',
 'astrometric_primary_flag',
 'astrometric_weight_al',
 'astrometric_pseudo_colour',
 'astrometric_pseudo_colour_error',
 'mean_varpi_factor_al',
 'astrometric_matched_observations',
 'visibility_periods_used',
 'astrometric_sigma5d_max',
 'frame_rotator_object_type',
 'matched_observations',
 'duplicated_source'

In [7]:
Gmag = df["phot_g_mean_mag"].values
GBmag = df["phot_bp_mean_mag"].values
GRmag = df["phot_rp_mean_mag"].values
# 2MASS bands
Jmag = df["j_m"].values
Hmag = df["h_m"].values
Kmag = df["ks_m"].values

# Other Important Data
plx = df["parallax"].values
Dist = 1000./plx
K_er = df["ks_msigcom"].values
J_er = df['j_msigcom']
H_er = df['h_msigcom']
G_flux = df["phot_g_mean_flux"]
G_flux_er = df["phot_g_mean_flux_error"]
GB_flux = df['phot_bp_mean_flux']
GB_flux_er = df['phot_bp_mean_flux_error']
GR_flux = df['phot_rp_mean_flux']
GR_flux_er = df['phot_rp_mean_flux_error']



In [94]:
Gmags = Gmag - 5 * np.log10(Dist) + 5
m = np.random.randint(1,len(Gmags))
mag1 = Gmag[m]
#mag2 = Gmag[3]
n = np.random.randint(1,len(Gmags))

flux1 = G_flux[m]
flux1_er = G_flux_er[m]
flux2 = G_flux[n]
flux2_er = G_flux_er[n]
def MagError(flux1,flux1_er,KnownMag,flux2,flux2_er):
    
    Mag = 2.5*np.log10(flux1/flux2) + KnownMag
    Mag_er = np.sqrt(((2.5/(np.log(10)*flux1))*flux1_er)**2 + ((-2.5/(np.log(10)*flux2))*flux2_er)**2)
    return Mag, Mag_er

Mag, Mag_er = MagError(flux1,flux1_er,mag1,flux2,flux2_er)

print(Gmag[n])
print(Mag)
print(Mag_er)
print(Gmags[n])

16.2617435455
16.2617444576
0.00120115353166
10.9222415465


In [51]:
c = np.random.randint(1,len(Gmags))
print(c)

841093
